# Libraries

In [ ]:
import numpy as np
import numpy.random as rnd
import numpy.linalg
import matplotlib.pyplot as plt
from numpy import linalg as LA
from numpy.random import normal
import scipy
from scipy.spatial.transform import Rotation as R
import plotly.graph_objects as go


In [ ]:


import numpy as np
from numpy import linalg as LA
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
import plotly.graph_objects as go


class Configuration:
    """Kagome Heisenberg model with correct neighbor mapping"""

    # --- voisins Kagome corrects ---
    # Chaque sous-site (0=A, 1=B, 2=C) a 4 voisins :
    # Donnés sous forme : (di, dj, s_voisin)
    KAGOME_NEIGHBORS = {
    0: [(0, 0, 1), (0, 0, 2), (0, -1, 1), (-1, 0, 2)],
    1: [(0, 0, 0), (0, 0, 2), (1, -1, 0), (1, 0, 2)],
    2: [(0, 0, 0), (0, 0, 1), (-1, 1, 0), (0, -1, 1)],
}


    def __init__(self, a, theta, Nx, Ny, J, T):
        self.a, self.theta, self.Nx, self.Ny, self.J, self.T = a, theta, Nx, Ny, J, T

        # Translation lattice vectors
        a1 = np.array([1, 0])
        a2 = np.array([1/2, np.sqrt(3)/2])
        
        unit_cell = np.array([
            [0, 0],
            [0.5, 0],
            [0.25, np.sqrt(3)/4]
        ])

        # Lattice coordinates
        x_idx, y_idx = np.meshgrid(np.arange(Nx), np.arange(Ny))
        self.lattice = (
            x_idx[:, :, None, None] * a2
            + y_idx[:, :, None, None] * a1
            + unit_cell[None, None, :, :]
        )

        # Random normalized spins
        np.random.seed(42)
        config = 2 * np.random.rand(Nx, Ny, 3, 3) - 1
        self.config = config / np.linalg.norm(config, axis=-1, keepdims=True)


    # --- Get sum of neighbors ---
    def sum_neighbors(self, i, j, s):
        S_sum = np.zeros(3)

        for di, dj, ss in self.KAGOME_NEIGHBORS[s]:
            ii = (i + di) % self.Nx
            jj = (j + dj) % self.Ny
            S_sum += self.config[ii, jj, ss, :]

        return S_sum


    # --- Energy of one spin ---
    def get_energy(self, i, j, s):
        S = self.config[i, j, s]
        S_neighbors = self.sum_neighbors(i, j, s)
        return self.J * np.dot(S, S_neighbors)


    # --- Total energy ---
    def total_energy(self):
        E = 0
        for i in range(self.Nx):
            for j in range(self.Ny):
                for s in range(3):
                    E += self.get_energy(i, j, s)
        return E / 2   # each pair counted twice


    # --- ΔE for spin update ---
    def delta_energy(self, i, j, s, new_spin):
        old_spin = self.config[i, j, s]
        neigh_sum = self.sum_neighbors(i, j, s)
        return self.J * np.dot(new_spin - old_spin, neigh_sum)


    # --- Standard Metropolis step ---
    def Monte_Carlo(self, Nf):
        beta = 1 / self.T

        for _ in range(Nf):
            i = np.random.randint(0, self.Nx)
            j = np.random.randint(0, self.Ny)
            s = np.random.randint(0, 3)

            # random rotation of spin
            axis = np.random.randn(3)
            axis /= np.linalg.norm(axis)
            
            # small angle update
            angle = 0.2  # small rotation amplitude
            rot = R.from_rotvec(angle * axis)
            
            new_spin = rot.apply(self.config[i, j, s])
            new_spin /= LA.norm(new_spin)

            dE = self.delta_energy(i, j, s, new_spin)

            if np.random.rand() < np.exp(-beta * dE):
                self.config[i, j, s] = new_spin
                
    def overrelaxation_step(self, Nf):
        for _ in range(Nf):
            i = np.random.randint(0, self.Nx)
            j = np.random.randint(0, self.Ny)
            s = np.random.randint(0, 3)

            S = self.config[i, j, s]
            H = self.sum_neighbors(i, j, s)

            if np.linalg.norm(H) < 1e-8:
                continue  # champ local nul, rien à faire

            H /= LA.norm(H)
            # rotation de 180° autour de H : conserve l'énergie locale
            S_par_H = np.dot(S, H) * H
            S_perp = S - S_par_H
            S_new = S_par_H - S_perp
            S_new /= LA.norm(S_new)

            self.config[i, j, s] = S_new


    # --- Monte Carlo + Overrelaxation sweep ---
    def Monte_Carlo_with_overrelaxation(self, Nf):
        # 1. Metropolis step
        self.Monte_Carlo(Nf)
        # 2. Overrelaxation step
        self.overrelaxation_step(Nf)

    # --- Energy monitoring for MC ---
    def run_MC_energy(self, Nsteps, Nf):
        energies = []
        for _ in range(Nsteps):
            self.Monte_Carlo(Nf)
            energies.append(self.total_energy())
        return np.array(energies)


    # --- Check spin normalization ---
    def verify_norm(self):
        print("Max norm =", LA.norm(self.config, axis=-1).max())


    # --- Plot lattice spins (unchanged) ---
    def display_config(self):
        position = np.reshape(self.lattice, (3*self.Nx*self.Ny, 2), order='C')
        arrows   = np.reshape(self.config,  (3*self.Nx*self.Ny, 3), order='C')

        x = position[:, 0]
        y = position[:, 1]
        z = np.zeros_like(x)

        Sx, Sy, Sz = arrows[:, 0], arrows[:, 1], arrows[:, 2]
        X_lines, Y_lines, Z_lines = [], [], []

        for xi, yi, zi, Sxi, Syi, Szi in zip(x, y, z, Sx, Sy, Sz):
            X_lines += [xi, xi + Sxi, None]
            Y_lines += [yi, yi + Syi, None]
            Z_lines += [zi, zi + Szi, None]

        fig = go.Figure()
        fig.add_trace(go.Scatter3d(x=X_lines, y=Y_lines, z=Z_lines, mode="lines"))
        fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode="markers"))

        fig.update_layout(scene=dict(aspectmode='data'))
        fig.show(renderer="browser")
        

# Infos about the simulation class

In [ ]:
J=100
Nx=3
Ny=3
Nf = 3 * Nx * Ny
theta=2*np.pi/3
a=1
T=0.1
config = Configuration(a, theta, Nx, Ny, J, T)
N_mc = 1000  # Monte Carlo steps for averaging


for step in range(N_mc):
    config.Monte_Carlo_with_overrelaxation(Nf*10000)
    


# --- NOW DISPLAY ---
config.display_config()



temperatures = np.linspace(0.005, 0.05, 5)  # temperature range
N_mc = 1000  # Monte Carlo steps for averaging

heat_capacity = []

for T in temperatures:
    # Initialize lattice
    lattice = Configuration(a, theta, Nx, Ny, J, T)
    
    energies = []
    
    for step in range(N_mc):
        lattice.Monte_Carlo(Nf=Nx*Ny*3)  # one MC step per spin
        if step >= 0.8 * N_mc :  #thermalization : append E only when it evolves around its mean value
            energies.append(lattice.total_energy())
    energies = np.array(energies)
    C = (np.mean(energies**2) - np.mean(energies)**2) / T**2
    heat_capacity.append(C/(3*Nx*Ny))

# Plot
plt.figure(figsize=(6,4))
plt.plot(temperatures, heat_capacity, 'o-', color='red')
plt.xlabel('Temperature T')
plt.ylabel('Heat Capacity C')
plt.title('Heat Capacity vs Temperature')
plt.grid(True)
plt.show()



energies = config.run_MC_energy(N_mc, Nf)

# ---------------------------
# Plot de l'énergie
# ---------------------------
plt.figure(figsize=(8, 5))
plt.plot(energies, '-b')
plt.xlabel("Pas de Monte Carlo")
plt.ylabel("Énergie totale")
plt.title("Évolution de l'énergie au cours des pas de Monte Carlo")
plt.grid(True)
plt.show()















